In [ ]:
import subprocess
import os

# CONFIGURATION 
vcf_input = "/mnt/jupiter/johnsonlab/Capstone_proj/results/BQSR/cohort_snps.vcf"
work_dir = "/mnt/jupiter/johnsonlab/Capstone_proj/results/BQSR/plink_analysis"
os.makedirs(work_dir, exist_ok=True)

plink_prefix = f"{work_dir}/cohort_plink"
stats_prefix = f"{work_dir}/cohort_stats"
filtered_prefix = f"{work_dir}/cohort_filtered"
final_vcf = f"{work_dir}/cohort_filtered_snps.vcf"

# STEP 1: Convert VCF to PLINK format
subprocess.run([
    "plink", "--vcf", vcf_input,
    "--allow-extra-chr",  # Accept non-standard chromosome names
    "--make-bed", "--out", plink_prefix
], check=True)

# STEP 2: Compute MAF and F_MISS
subprocess.run([
    "plink", "--bfile", plink_prefix,
    "--allow-extra-chr",  
    "--freq",
    "--missing",
    "--out", stats_prefix
], check=True)

# STEP 3: Filter SNPs
subprocess.run([
    "plink", "--bfile", plink_prefix,
    "--allow-extra-chr",  
    "--maf", "0.1",
    "--geno", "0.05",
    "--make-bed",
    "--out", filtered_prefix
], check=True)

# STEP 4: Export to VCF
subprocess.run([
    "plink", "--bfile", filtered_prefix,
    "--allow-extra-chr",  
    "--recode", "vcf",
    "--out", final_vcf.replace(".vcf", "")
], check=True)

print(f"Done! Final VCF: {final_vcf}")
